<a href="https://colab.research.google.com/github/LizaPiya/IPH-DS-LLM-Projects/blob/main/LLM_With_G35_Data_14th_August_WORKS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Import the necessary module for downloading files
from google.colab import files

# **Downloading Necessary Data**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
Top_Notes='/content/drive/MyDrive/Fahmida Liza Piya/top_notes.csv'
Top_Notes=pd.read_csv(Top_Notes)
Top_Notes.head(5)

,patientpseudonymtoken,birthyear,age,Gender,Race,nextgengroupid,referencedatetime,clinicid,reportingdate,reportingenterpriseid,...,diagnosisdate,onsetdate,tablename,fieldname,chartnotes,word_count,sentence_lengths,num_sentences,total_words,avg_sentence_length
0,b2PdD7imRVG9WB14DcvdqgpKYWu8TBpv6uWoMPe41ALfJr...,1964,60,F,C,21377,202104231018,998,20210401,998,...,202104231018,20210423,HOPI_,DETAIL1,The symptoms began 29 years ago and generally ...,1507,"[10, 7, 4, 5, 10, 22, 14, 5, 7, 9, 19, 10, 16,...",89,1507,16.932584
1,RH2r2wG3JH7gP+F8B+hPqacn9zlUickN3rJGzLGzK6bfsZ...,1967,57,M,A,17392,202404080920,2375,20240401,2375,...,202404080920,20240408,HOPI_,DETAIL,This is a 55 year old man who requested to es...,1158,"[0, 20, 22, 11, 21, 34, 19, 17, 6, 18, 23, 22,...",56,1158,20.678571
2,PN5CtDsE3eH/eDR64Rl5OndIEVVB9e1MZBHq7kwY0kEaUP...,1945,79,F,C,38068,201706061000,1244,20170601,1244,...,201706061000,20170606,ASSESSMENT_IMPRESSION_PLAN_,TXT_DESCRIPTION,"1. The patient is a 76-year-old, right-handed,...",762,"[1, 29, 14, 16, 13, 4, 13, 9, 8, 10, 8, 8, 8, ...",52,762,14.653846
3,dqTRLgZJqfqnWwxMuyBncAcwdd8ARyrvkIf4HhsunjnF0b...,1979,45,M,A,79009,202210061145,745,20221001,745,...,202210061145,20221006,HOPI_,DETAIL,This is a very pleasant 44-year old Caucasian...,736,"[0, 25, 11, 19, 16, 11, 14, 20, 20, 21, 9, 24,...",51,736,14.431373
4,dqTRLgZJqfqnWwxMuyBncAcwdd8ARyrvkIf4HhsunjnF0b...,1979,45,M,C,79009,201904251030,745,20190401,745,...,201904251030,20190425,HOPI_,DETAIL1,This is a very pleasant 44-year old Caucasian ...,736,"[25, 11, 19, 16, 11, 14, 20, 20, 21, 9, 24, 11...",50,736,14.720000


# **Importing Libraries for LLM**

In [4]:
!pip install langchain langchain_community langchain_chroma

In [5]:
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [6]:
# prompt: #load this csv file for langchain

loader = CSVLoader(file_path='/content/drive/MyDrive/Fahmida Liza Piya/top_notes.csv')
data = loader.load()


In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)  # Adjust chunk size as needed
notes_texts = []
for doc in data:
  split_texts = text_splitter.split_text(doc.page_content)
  notes_texts.extend(split_texts)
  print(f"Number of chunks for this document: {len(split_texts)}")
  for chunk in split_texts:
      print(f"Chunk length: {len(chunk)}")
  print("-" * 30)

Streaming output truncated to the last 5000 lines.
------------------------------
Number of chunks for this document: 6
Chunk length: 429
Chunk length: 498
Chunk length: 498
Chunk length: 498
Chunk length: 163
Chunk length: 191
------------------------------
Number of chunks for this document: 6
Chunk length: 405
Chunk length: 497
Chunk length: 495
Chunk length: 499
Chunk length: 128
Chunk length: 162
------------------------------
Number of chunks for this document: 6
Chunk length: 415
Chunk length: 493
Chunk length: 498
Chunk length: 491
Chunk length: 275
Chunk length: 177
------------------------------
Number of chunks for this document: 6
Chunk length: 417
Chunk length: 498
Chunk length: 497
Chunk length: 496
Chunk length: 192
Chunk length: 198
------------------------------
Number of chunks for this document: 6
Chunk length: 399
Chunk length: 491
Chunk length: 494
Chunk length: 495
Chunk length: 265
Chunk length: 155
------------------------------
Number of chunks for this documen

In [8]:
!pip install sentence-transformers langchain
!pip install langchain_experimental
!pip install faiss-cpu

In [9]:
# Import RAG from langchain_experimental
from langchain.chains import RetrievalQA
from langchain.embeddings import SentenceTransformerEmbeddings
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from faiss import IndexFlatL2

In [24]:
!pip install transformers --upgrade

In [10]:
from langchain.embeddings import SentenceTransformerEmbeddings
from sentence_transformers import SentenceTransformer

# Initialize the Sentence Transformer model
model_name = 'all-MiniLM-L6-v2'
embedding_function = SentenceTransformerEmbeddings(model_name=model_name)


# Perform embeddings
document_embeddings = embedding_function.embed_documents(notes_texts)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10

In [11]:
# Display the embeddings
for i, embedding in enumerate(document_embeddings):
  print(f"Text chunk {i}: {notes_texts[i]}")
  print(f"Embedding {i}: {embedding[:10]}... (truncated for display)\n")

Streaming output truncated to the last 5000 lines.
total_words: 136
avg_sentence_length: 6.476190476190476
Embedding 4782: [0.07806210964918137, -0.01516522467136383, 0.014213080517947674, 0.013318359851837158, -0.037414468824863434, 0.0748603343963623, 0.006764878518879414, 0.09206505864858627, -0.020216306671500206, 0.029620682820677757]... (truncated for display)

Text chunk 4783: patientpseudonymtoken: FumsHiX0uWNsL6FgVc5fHuoMcnhLKQpadI9l1BaGhCdRSdmoECFatNdGsp2ME1ATbQSV5DPpFUecBYhyKT2O5w
birthyear: 1945
age: 79
Gender: F
Race: C
nextgengroupid: 27052
referencedatetime: 202103231330
clinicid: 666
reportingdate: 20210301
reportingenterpriseid: 666
encounterid: 187705
encounterdate: 20210323
diagnosisdate: 202103231330
onsetdate: 20210323
tablename: ASSESSMENT_IMPRESSION_PLAN_
fieldname: TXT_DESCRIPTION
Embedding 4783: [0.023034926503896713, 0.09041041135787964, -0.04786089435219765, 0.03410879522562027, -0.033551521599292755, -0.00671611912548542, -0.0017906036227941513, 0.0502918325

In [12]:
# Step to use FAISS for indexing and retrieval
# Create an FAISS index
embedding_dim = len(document_embeddings[0])
index = IndexFlatL2(embedding_dim)


In [13]:
# Initialize the FAISS vector store
vectorstore = FAISS(
  embedding_function=embedding_function,
  index=index,
  docstore=InMemoryDocstore(),
  index_to_docstore_id={i: str(i) for i in range(len(notes_texts))}
)


In [14]:
# Add embeddings to the vector store
vectorstore.add_embeddings(list(zip(notes_texts, document_embeddings)))

['d22009f7-4baa-43fd-9747-66b3e16f905a',
 '635c61f2-5272-4850-9e5d-f2f007d6b5bc',
 'c8addf3c-6a44-4694-bcf4-5890b2111aa1',
 '00372446-697b-4067-b8f6-94c6a136a4c0',
 'b24b698d-5ef4-4b6c-a92d-c5a03c902f57',
 '1f2f1048-d89f-42c9-9585-ce0cd4207ac2',
 'b1816295-0af6-485c-baf6-2bc93fc132e1',
 '5503456b-9185-48aa-8b75-63764ee90924',
 '137b68f7-bf36-472c-be2c-34a18413289d',
 '1e14c1f8-bb40-45f2-abe0-2f734f83ceda',
 '8e81cb78-a9ee-4503-95b7-dedb30e4ff44',
 '3dace8df-9055-40ea-a5ed-f00c3caa4cf7',
 '161ccaf9-ce13-4c16-80e9-b3f11291701b',
 'f1aa7b36-74e4-4619-8197-d3c83b66b71d',
 '24af4076-67da-4c16-a7b2-94925b21a736',
 '673e267e-d3b5-47b8-8d90-40cbd59a6bf0',
 '85e2db3f-b1fd-47c3-88fe-2e082635deff',
 '5798510d-c9a8-4d82-8766-043a001e51d0',
 '0b00cceb-096f-40b9-9b37-11a8a35bb643',
 'fea5e538-bf32-462b-8045-19a5253cd056',
 '171c3bf9-08ec-45b5-829c-523a422564ef',
 '9a536dfc-8272-4578-aa11-f5b2afc7dded',
 '2a008ef4-eb83-4764-ba28-a0a245d61ad1',
 '5740ba0b-f82c-4525-b7d0-734a6165d6bf',
 'b9928059-aba5-

In [15]:
!pip install --upgrade langchain
!pip install FAISS --gpu


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --gpu


In [16]:
import json
import torch
from transformers import (AutoTokenizer,
                        AutoModelForCausalLM,
                        BitsAndBytesConfig,
                        pipeline)
import numpy as np

In [17]:
!pip install accelerate==0.32.1
!pip install bitsandbytes==0.43.1

In [18]:
HF_TOKEN = "hf_"
model_name = "meta-llama/Llama-Guard-3-8B"

In [19]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)


In [20]:
model = AutoModelForCausalLM.from_pretrained(
   model_name,
   device_map="auto",
   quantization_config=bnb_config,
   token=HF_TOKEN,
)


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [21]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                        token=HF_TOKEN)
tokenizer.pad_token = tokenizer.eos_token


tokenizer_config.json:   0%|          | 0.00/52.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [22]:
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128
)

In [23]:
def get_response(prompt):
  sequences = text_generator(prompt)
  gen_text = sequences[0]["generated_text"]
  return gen_text

In [25]:
from transformers import pipeline

# Initialize the text generation pipeline
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Define the get_response function with the refined prompt
def get_response(context):
    # Refined prompt with structure
    prompt_template = """
    [INST]≪SYS≫You are a helpful and knowledgeable assistant with expertise in neurology.≪/SYS≫
    Use the provided context to fill in the medications provided to Multiple Sclerosis patients with 10 words or less. If you don't know the answer, do not add extra information.

    Context: {context}
    Key Features Template:
    1. [Medication_1]
    2. [Medication_2]
    3. [Medication_3]
    4. [Medication_4]
    Filled template:
    [/INST]
    """

    # Format the prompt with the context
    prompt = prompt_template.format(context=context)

    # Generate the response with max_new_tokens instead of max_length
    sequences = text_generator(prompt, max_new_tokens=50, truncation=True)
    gen_text = sequences[0]["generated_text"]

    return gen_text

# Example context (replace this with your actual input context)
context = "The symptoms began 29 years ago and generally lasts varies. The symptoms are reported as being moderate. The symptoms occur daily. The location is see below. She states the symptoms are chronic and are fairly controlled. ^^^^Initial History:^^The patient is a 42-year-old woman with long-standing history of relapsing remitting multiple sclerosis and secondary optic neuropathy. She has been using Avonex on a regular basis and has had no exacerbations. She has no double vision. She has no blindness in the eye. She has been taking Fioricet as needed for headaches. For her multiple sclerosis, she has been using Avonex injections on a weekly basis and amantadine for her fatigue. She also uses Elavil 50 mg at bedtime for headaches. She has no other symptoms.^^^^Multiple Sclerosis ^^Treatment with steroids since last visit: none^^Current disease modifying therapy: Date Started: 03/14/2004 Avonex^^MS related symptomatic meds: Amantadine^^Since last visit: ^^Memory: normal^^Energy/fatigue: general tiredness^^Dizziness: minimal lightheaded^^Vertigo: mild^^Facial pain: none^^^^The patient also has history of headaches. Onset: 8 Years Ago. Severity: mild-moderate. It occurs intermittently. The problem is unchanged. Location was entire head and occipital. There was radiation to posterior and neck. The patient describes it as dull, pressure and squeezing. Timing of headache: daytime. Context: family history of migraine and history of migraine. Symptom is aggravated by bright lights, noise and stress. Relieving factors include prescription drugs and sleep. Associated symptoms include dizziness and vertigo. Pertinent negatives include blurred vision, clear sinus drainage, double vision, fever, hemianopsia left, hemianopsia right, loss of consciousness, memory impairment, nausea, neck stiffness, neurological symptoms, performance changes, personality change, phonophobia, photophobia, scintillations, scotoma, URI like symptoms, vision loss left, vision loss right, visual aura and vomiting. Additional information: She uses the Elavil 50mg and Fioricet prn.^^^^Interval History:^^2/9/2021:^^The patient states that she was admitted to Southside Hospital on November 9, 2020 after she developed left facial pain and swelling. She was found to have left facial cellulitis and was admitted there for 3 to 4 days and placed on IV antibiotics. She is feeling much better. However she complains that her headaches have been increasing over the past few months. She is currently taking Elavil 50 mg at bedtime as well as Fioricet as needed. As pertains to her multiple sclerosis she has been stable on the Avonex injections that she takes monthly. She has had no untoward side effects. She also takes amantadine. She has no other complaints.^^^^7/8/2021:^^The patient notes that she has been getting intermittent shocking type sensations that travels from her mid back down into her legs. That has been concerning her. She also complains of increasing headaches. The headaches are described a combination of wavy colors in front of her eyes followed by throbbing headaches. She has used Fioricet more frequently which has not helped. She takes Elavil 70 mg at bedtime. She was previously tried on Imitrex, Zomig and the other triptans which have not helped her. She has no weakness. She does have some neck pain. For her multiple sclerosis she has been taking Avonex. She also uses amantadine 100 mg twice a day for fatigue. The patient is seen regularly by the neuro-ophthalmologist Dr. Edward Barron.^^^^10/12/2021:^^The patient states that she has been experiencing increased persistent headaches. She has had difficulty sleeping at night because of that. She tried the Nurtec ODT which has not been helping. She has been taking amitriptyline 70 mg at bedtime. She feels is not helping very much. She also uses Fioricet as needed. In addition to the above she has had multiple joint pains and has been seen by Dr. Kleiner the rheumatologist. She has been Avonex on a regular basis for her MS.^^^^2/24/2022:^^The patient is a 57-year-old woman with longstanding history of multiple sclerosis. She states that she recently had right eye laser surgery for retinal tear. She has been followed by Dr. Edward Barron the neuro-ophthalmologist. She has been noticing that she has been experiencing left-sided headaches and some pain in the eye over the past few months. She had been on tapering dose of prednisone and was supposed to increase amitriptyline to 80 mg. However she has been taking only 70 mg at bedtime. The patient notes that barometric changes in the weather also trigger headaches. She uses Fioricet in between. She has been complaining of increased numbness of the left hand. She had right carpal tunnel release surgery which had helped her previously.^^^^5/24/2022:^^The patient states that she has been doing well overall. She has been followed by Dr. Edward Barron the neuro-ophthalmologist. Her headaches are variable and are partly controlled by the amitriptyline 80 mg at bedtime. She uses Fioricet as needed for recurrent breakthrough headaches. In addition for her MS she uses Avonex as well. She has been using amantadine twice a day for her fatigue. She has been complaining of sweating in her scalp. She has not yet seen her gynecologist.^^^^11/10/2022:^^The patient states that she was given Paxil of it after her boyfriend and daughter were found to be COVID positive. She developed an allergic reaction to that and had swelling and rash throughout her lower extremities. She was seen at Good Samaritan Hospital emergency room and was given some medications and sent home. She notes that she has been losing her balance and has fallen down a few times since then. She also has been complaining of headaches worse in the left retro-orbital region for the past 2 weeks. The Fioricet is not helping and the Elavil has not been helping as well. She has no double vision although she does complain of some blurred vision for which she has been seeing her ophthalmologist on a regular basis. She has no dizziness. She sees Dr. Barron the neuro-ophthalmologist on a regular basis.^^^^Diagnostic Studies:^^MRI scan of the brain with contrast and without gadolinium done on 11/24/13 and revealed findings compatible with the patient's diagnosis of multiple sclerosis and stable chronic periventricular white matter plaques. There is no abnormal area of enhancement.^^Brainstem auditory evoked potential studies done on 12/5/13 were normal.^^Otoacoustic admissions studies done on 12/5/13 revealed peripheral hearing loss worse on the right when compared to the left.^^Somatosensory evoked potential studies of the upper and lower extremities done on the same date to my review are normal.^^Visual evoked potential studies done on 12/5/13 showed mild-to-moderate left optic neuropathy which is unchanged from the prior study.^^^^MRI scan of the thoracic spine done on October 28, 2015 is normal.^^Blood tests done by her rheumatologist on October 13, 2015 revealed normal CBC and CMP. Her B12 and folic acid levels were normal. She had normal CPK levels. Her serum electrophoresis was also normal. However the patient had a positive anticardiolipin antibody titers as well as elevated ENA.^^^^MRI scan of the brain done without and with contrast on November 30, 2016 revealed no significant interval change. He has multiple plaques and in no innumerable Dawson's fingers which are consistent with a history of multiple sclerosis.^^Brain stem auditory evoked potential studies done on December 20, 2016 were normal.^^Visual evoked potential studies done on the same day revealed findings consistent with mild to moderate left optic neuropathy which remains unchanged when compared to the study done in July 2015.^^^^MRI scan of the brain done without and with contrast on March 15, 2018 revealed no interval change in the moderate demyelinating plaque burden of multiple sclerosis. There are no new enhancing or nonenhancing lesions.^^Brainstem auditory evoked potential studies done on April 10, 2018 are normal.^^Somatosensory evoked potential studies of the upper and lower extremities done on the same day are also normal.^^Visual evoked potential studies done on April 10, 2018 are abnormal indicative of mild left optic neuropathy. Compared to the previous study done in December 2016 there has been mild improvement on the left.^^^^Blood tests done by her rheumatologist Dr. Kleiner on May 31, 2019 revealed beta 2 glycoprotein antibodies, immunoglobulin panels, B12 and folic acid levels, thyroid function test, sedimentation rate, CMP, LDH, magnesium levels, phosphorus levels, complement levels, C-reactive protein, immunofixation CBC, anticardiolipin antibodies ,which were all normal.^^Blood tests done on July 10, 2019 revealed normal CMP and CBC with slightly low CD8 lymphocytes.^^^^EMG and nerve conduction studies of the upper extremities done on August 28, 2019 revealed severe right carpal tunnel syndrome and moderately severe left carpal tunnel syndrome.^^MRI scan of the brain done without contrast on July 21, 2021 reveals no change in the patient's multiple sclerosis when compared to the prior study.^^MRI scan of the cervical spine done on July 14, 2021 reveals single demyelinating lesion in the cervical cord at C5-6.^^MRI scan of the thoracic spine done on the same day is normal.^^Brainstem auditory evoked potential studies done on August 25, 2021 are normal. Somatosensory evoked potential studies done on the same day are abnormal with central conduction delay.^^Visual evoked response study done on the same day shows mild to moderate bilateral optic neuropathy which is mildly worsened^^^^MR angiography of the brain done without contrast on 10/9/2021 is normal."
# Get the structured response
response = get_response(context)
print(response)



    [INST]≪SYS≫You are a helpful and knowledgeable assistant with expertise in neurology.≪/SYS≫
    Use the provided context to fill in the medications provided to Multiple Sclerosis patients with 10 words or less. If you don't know the answer, do not add extra information.

    Context: The symptoms began 29 years ago and generally lasts varies. The symptoms are reported as being moderate. The symptoms occur daily. The location is see below. She states the symptoms are chronic and are fairly controlled. ^^^^Initial History:^^The patient is a 42-year-old woman with long-standing history of relapsing remitting multiple sclerosis and secondary optic neuropathy. She has been using Avonex on a regular basis and has had no exacerbations. She has no double vision. She has no blindness in the eye. She has been taking Fioricet as needed for headaches. For her multiple sclerosis, she has been using Avonex injections on a weekly basis and amantadine for her fatigue. She also uses Elavil 50 mg a

In [27]:
from transformers import pipeline

# Initialize the text generation pipeline
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Define the get_response function with the refined prompt
def get_response(ms_note):
    # Refined prompt with structure
    prompt_template = """
    You are a helpful assistant.
    Please fill in the following patient summary template based on the given context.


    Context: {ms_note}

    Summary Template:
    Age: [Fill in Age]
    Gender: [Fill in Gender]
    Weight: [Fill in Weight]
    BMI: [Fill in BMI]
    Weight history: [Fill in Weight history]
    Medical history: [Fill in Medical history]
    List risk factors causing multiple sclerosis: [Fill in risk factors]
    List multiple sclerosis interventions or recommendations: [Fill in interventions]

    Filled template:
    """

    # Format the prompt with the context
    prompt = prompt_template.format(ms_note=ms_note)

    # Generate the response with max_new_tokens instead of max_length
    sequences = text_generator(prompt, max_new_tokens=150, truncation=True)
    gen_text = sequences[0]["generated_text"]

    return gen_text

# Example context focused on Multiple Sclerosis (replace this with your actual input context)
ms_note = "The symptoms began 29 years ago and generally lasts varies. The symptoms are reported as being moderate. The symptoms occur daily. The location is see below. She states the symptoms are chronic and are fairly controlled. ^^^^Initial History:^^The patient is a 42-year-old woman with long-standing history of relapsing remitting multiple sclerosis and secondary optic neuropathy. She has been using Avonex on a regular basis and has had no exacerbations. She has no double vision. She has no blindness in the eye. She has been taking Fioricet as needed for headaches. For her multiple sclerosis, she has been using Avonex injections on a weekly basis and amantadine for her fatigue. She also uses Elavil 50 mg at bedtime for headaches. She has no other symptoms.^^^^Multiple Sclerosis ^^Treatment with steroids since last visit: none^^Current disease modifying therapy: Date Started: 03/14/2004 Avonex^^MS related symptomatic meds: Amantadine^^Since last visit: ^^Memory: normal^^Energy/fatigue: general tiredness^^Dizziness: minimal lightheaded^^Vertigo: mild^^Facial pain: none^^^^The patient also has history of headaches. Onset: 8 Years Ago. Severity: mild-moderate. It occurs intermittently. The problem is unchanged. Location was entire head and occipital. There was radiation to posterior and neck. The patient describes it as dull, pressure and squeezing. Timing of headache: daytime. Context: family history of migraine and history of migraine. Symptom is aggravated by bright lights, noise and stress. Relieving factors include prescription drugs and sleep. Associated symptoms include dizziness and vertigo. Pertinent negatives include blurred vision, clear sinus drainage, double vision, fever, hemianopsia left, hemianopsia right, loss of consciousness, memory impairment, nausea, neck stiffness, neurological symptoms, performance changes, personality change, phonophobia, photophobia, scintillations, scotoma, URI like symptoms, vision loss left, vision loss right, visual aura and vomiting. Additional information: She uses the Elavil 50mg and Fioricet prn.^^^^Interval History:^^2/9/2021:^^The patient states that she was admitted to Southside Hospital on November 9, 2020 after she developed left facial pain and swelling. She was found to have left facial cellulitis and was admitted there for 3 to 4 days and placed on IV antibiotics. She is feeling much better. However she complains that her headaches have been increasing over the past few months. She is currently taking Elavil 50 mg at bedtime as well as Fioricet as needed. As pertains to her multiple sclerosis she has been stable on the Avonex injections that she takes monthly. She has had no untoward side effects. She also takes amantadine. She has no other complaints.^^^^7/8/2021:^^The patient notes that she has been getting intermittent shocking type sensations that travels from her mid back down into her legs. That has been concerning her. She also complains of increasing headaches. The headaches are described a combination of wavy colors in front of her eyes followed by throbbing headaches. She has used Fioricet more frequently which has not helped. She takes Elavil 70 mg at bedtime. She was previously tried on Imitrex, Zomig and the other triptans which have not helped her. She has no weakness. She does have some neck pain. For her multiple sclerosis she has been taking Avonex. She also uses amantadine 100 mg twice a day for fatigue. The patient is seen regularly by the neuro-ophthalmologist Dr. Edward Barron.^^^^10/12/2021:^^The patient states that she has been experiencing increased persistent headaches. She has had difficulty sleeping at night because of that. She tried the Nurtec ODT which has not been helping. She has been taking amitriptyline 70 mg at bedtime. She feels is not helping very much. She also uses Fioricet as needed. In addition to the above she has had multiple joint pains and has been seen by Dr. Kleiner the rheumatologist. She has been Avonex on a regular basis for her MS.^^^^2/24/2022:^^The patient is a 57-year-old woman with longstanding history of multiple sclerosis. She states that she recently had right eye laser surgery for retinal tear. She has been followed by Dr. Edward Barron the neuro-ophthalmologist. She has been noticing that she has been experiencing left-sided headaches and some pain in the eye over the past few months. She had been on tapering dose of prednisone and was supposed to increase amitriptyline to 80 mg. However she has been taking only 70 mg at bedtime. The patient notes that barometric changes in the weather also trigger headaches. She uses Fioricet in between. She has been complaining of increased numbness of the left hand. She had right carpal tunnel release surgery which had helped her previously.^^^^5/24/2022:^^The patient states that she has been doing well overall. She has been followed by Dr. Edward Barron the neuro-ophthalmologist. Her headaches are variable and are partly controlled by the amitriptyline 80 mg at bedtime. She uses Fioricet as needed for recurrent breakthrough headaches. In addition for her MS she uses Avonex as well. She has been using amantadine twice a day for her fatigue. She has been complaining of sweating in her scalp. She has not yet seen her gynecologist.^^^^11/10/2022:^^The patient states that she was given Paxil of it after her boyfriend and daughter were found to be COVID positive. She developed an allergic reaction to that and had swelling and rash throughout her lower extremities. She was seen at Good Samaritan Hospital emergency room and was given some medications and sent home. She notes that she has been losing her balance and has fallen down a few times since then. She also has been complaining of headaches worse in the left retro-orbital region for the past 2 weeks. The Fioricet is not helping and the Elavil has not been helping as well. She has no double vision although she does complain of some blurred vision for which she has been seeing her ophthalmologist on a regular basis. She has no dizziness. She sees Dr. Barron the neuro-ophthalmologist on a regular basis.^^^^Diagnostic Studies:^^MRI scan of the brain with contrast and without gadolinium done on 11/24/13 and revealed findings compatible with the patient's diagnosis of multiple sclerosis and stable chronic periventricular white matter plaques. There is no abnormal area of enhancement.^^Brainstem auditory evoked potential studies done on 12/5/13 were normal.^^Otoacoustic admissions studies done on 12/5/13 revealed peripheral hearing loss worse on the right when compared to the left.^^Somatosensory evoked potential studies of the upper and lower extremities done on the same date to my review are normal.^^Visual evoked potential studies done on 12/5/13 showed mild-to-moderate left optic neuropathy which is unchanged from the prior study.^^^^MRI scan of the thoracic spine done on October 28, 2015 is normal.^^Blood tests done by her rheumatologist on October 13, 2015 revealed normal CBC and CMP. Her B12 and folic acid levels were normal. She had normal CPK levels. Her serum electrophoresis was also normal. However the patient had a positive anticardiolipin antibody titers as well as elevated ENA.^^^^MRI scan of the brain done without and with contrast on November 30, 2016 revealed no significant interval change. He has multiple plaques and in no innumerable Dawson's fingers which are consistent with a history of multiple sclerosis.^^Brain stem auditory evoked potential studies done on December 20, 2016 were normal.^^Visual evoked potential studies done on the same day revealed findings consistent with mild to moderate left optic neuropathy which remains unchanged when compared to the study done in July 2015.^^^^MRI scan of the brain done without and with contrast on March 15, 2018 revealed no interval change in the moderate demyelinating plaque burden of multiple sclerosis. There are no new enhancing or nonenhancing lesions.^^Brainstem auditory evoked potential studies done on April 10, 2018 are normal.^^Somatosensory evoked potential studies of the upper and lower extremities done on the same day are also normal.^^Visual evoked potential studies done on April 10, 2018 are abnormal indicative of mild left optic neuropathy. Compared to the previous study done in December 2016 there has been mild improvement on the left.^^^^Blood tests done by her rheumatologist Dr. Kleiner on May 31, 2019 revealed beta 2 glycoprotein antibodies, immunoglobulin panels, B12 and folic acid levels, thyroid function test, sedimentation rate, CMP, LDH, magnesium levels, phosphorus levels, complement levels, C-reactive protein, immunofixation CBC, anticardiolipin antibodies ,which were all normal.^^Blood tests done on July 10, 2019 revealed normal CMP and CBC with slightly low CD8 lymphocytes.^^^^EMG and nerve conduction studies of the upper extremities done on August 28, 2019 revealed severe right carpal tunnel syndrome and moderately severe left carpal tunnel syndrome.^^MRI scan of the brain done without contrast on July 21, 2021 reveals no change in the patient's multiple sclerosis when compared to the prior study.^^MRI scan of the cervical spine done on July 14, 2021 reveals single demyelinating lesion in the cervical cord at C5-6.^^MRI scan of the thoracic spine done on the same day is normal.^^Brainstem auditory evoked potential studies done on August 25, 2021 are normal. Somatosensory evoked potential studies done on the same day are abnormal with central conduction delay.^^Visual evoked response study done on the same day shows mild to moderate bilateral optic neuropathy which is mildly worsened^^^^MR angiography of the brain done without contrast on 10/9/2021 is normal."
# Get the structured summary
response = get_response(ms_note)
print(response)



    You are a helpful assistant.
    Please fill in the following patient summary template based on the given context.
    

    Context: The symptoms began 29 years ago and generally lasts varies. The symptoms are reported as being moderate. The symptoms occur daily. The location is see below. She states the symptoms are chronic and are fairly controlled. ^^^^Initial History:^^The patient is a 42-year-old woman with long-standing history of relapsing remitting multiple sclerosis and secondary optic neuropathy. She has been using Avonex on a regular basis and has had no exacerbations. She has no double vision. She has no blindness in the eye. She has been taking Fioricet as needed for headaches. For her multiple sclerosis, she has been using Avonex injections on a weekly basis and amantadine for her fatigue. She also uses Elavil 50 mg at bedtime for headaches. She has no other symptoms.^^^^Multiple Sclerosis ^^Treatment with steroids since last visit: none^^Current disease modifying t